<a target="_blank" href="https://colab.research.google.com/github/amir-meshkini/protein_deltaG_classifier/blob/main/01_data_preprocessing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# ΔG Prediction: Preprocessing Notebook 

In this notebook the first 200000 record of the PSPD dataset are tokenized and vectorized, using the Facebook's `EMS2 8M` PLP (Protein Language Processing) model. 

The vectorized data meant to be used later as the training data for our ΔG classifier prediction model.

### Importing the packages 

A full list of required packages and their corresponding versions is provided in the `requirements.txt` file.

In [1]:
import numpy as np 
import pandas as pd 
import math
import tensorflow as tf 
from transformers import AutoTokenizer, TFEsmModel

### Importing the dataset

Here, the `smallpspd.csv` is simply the first 200000 rows of the [original PSPD dataset](https://huggingface.co/datasets/benchang323/protein-stability-prediction) ([licensed by MIT](https://choosealicense.com/licenses/mit/)) which has over 4 million records.

In [2]:
pspd = pd.read_csv('smallpspd.csv') 
pspd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  200000 non-null  int64  
 1   aa_seq      200000 non-null  object 
 2   deltaG      200000 non-null  float64
 3   deltaG_bin  200000 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [3]:
sequences = list(pspd['aa_seq'])

# A sample of the amino acid sequences
sequences[0]

'MKIFVKTLTGKTITLEVEPSDTIENVKAKIQDEEGIPPDQQRLIFAGKKLEDGRTLTDYSIQKESTLHLVLR'

### Encoding the labels

The ΔG values of this dataset lies between -10 to +15 Kcal/mol. therefore 26 different classes are considered that correspond to 26 different intervals of integers. 

In [4]:
# Giving each interval a label (integers from 0 to 25)
delg_range = pspd['deltaG_bin'] 
label, interval = pd.factorize(delg_range) 
delg_range_labels = pd.DataFrame({'deltaG (Kcal/mol)': delg_range, 'labels': label}) 

delg_range_labels.head(10)

,deltaG (Kcal/mol),labels
0,8 to 9,0
1,5 to 6,1
2,0 to 1,2
3,-4 to -3,3
4,5 to 6,1
5,3 to 4,4
6,3 to 4,4
7,4 to 5,5
8,1 to 2,6
9,12 to 13,7


In [5]:
code_to_value = dict(enumerate(interval))
code_to_value

{0: '8 to 9',
 1: '5 to 6',
 2: '0 to 1',
 3: '-4 to -3',
 4: '3 to 4',
 5: '4 to 5',
 6: '1 to 2',
 7: '12 to 13',
 8: '-5 to -4',
 9: '-2 to -1',
 10: '10 to 11',
 11: '11 to 12',
 12: '-9 to -8',
 13: '-7 to -6',
 14: '-8 to -7',
 15: '-10 to -9',
 16: '13 to 14',
 17: '-3 to -2',
 18: '2 to 3',
 19: '-6 to -5',
 20: '9 to 10',
 21: '7 to 8',
 22: '15 to 16',
 23: '14 to 15',
 24: '6 to 7',
 25: '-1 to 0'}

In [23]:
y_data = delg_range_labels['labels'].to_numpy()

### Using the pre-trained models 

After initializing the tokenizer and the EMS model, A function is defined to feed the data to the model in chunks, in order to avoid getting the `ResourceExhaustedError` 

> **Note:** If you wish to rerun this code on your local machine, you might be having to adjust the patch size, considering your own RAM (or VRAM if you'r using CUDA) to avoid getting the mentioned error.

In [7]:
# using EMS2 model with 8 million parameters
model_name = "facebook/esm2_t6_8M_UR50D"

# Initializing the tokenizer and the EMS model
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = TFEsmModel.from_pretrained(model_name)

c:\Users\amesh\anaconda3\envs\cvproj\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFEsmModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'esm.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'esm.embeddings.position_embeddings.weight']
- This IS expected if you are initializing TFEsmModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEsmModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertFo

In [8]:
def get_embeddings_in_batches(sequence_list, batch_size=8):
    all_vectors = []
    
    total_batches = math.ceil(len(sequence_list) / batch_size)
    print(f"Processing {len(sequence_list)} sequences in {total_batches} batches...")

    for i in range(0, len(sequence_list), batch_size):
        # Create the batch
        batch_sequences = sequence_list[i : i + batch_size]
        
        # Tokenize ONLY this batch
        # max_length=1024 ensures super long proteins don't crash memory
        inputs = tokenizer(batch_sequences, return_tensors="tf", padding=True, truncation=True, max_length=1024)
        
        # Inference
        outputs = model(inputs, training=False)
        
        # Pooling (same logic as before)
        token_embeddings = outputs.last_hidden_state
        attention_mask = inputs['attention_mask']
        
        mask = tf.cast(tf.expand_dims(attention_mask, -1), tf.float32)
        sum_embeddings = tf.reduce_sum(token_embeddings * mask, axis=1)
        sum_mask = tf.reduce_sum(mask, axis=1)
        
        # Calculate mean vectors for this batch
        batch_vectors = sum_embeddings / tf.maximum(sum_mask, 1e-9)
        
        # Add to the main list
        all_vectors.append(batch_vectors.numpy())
        
        print(f"Batch {i//batch_size + 1}/{total_batches} done.")

    # Combine all batches into one big matrix
    return np.concatenate(all_vectors, axis=0)

# 4. Run extraction
# Try batch_size=4 or batch_size=2 if you still get errors
X_data = get_embeddings_in_batches(sequences, batch_size=1000)

print(f"Final shape: {X_data.shape}")

Processing 200000 sequences in 200 batches...
Batch 1/200 done.
Batch 2/200 done.
Batch 3/200 done.
Batch 4/200 done.
Batch 5/200 done.
Batch 6/200 done.
Batch 7/200 done.
Batch 8/200 done.
Batch 9/200 done.
Batch 10/200 done.
Batch 11/200 done.
Batch 12/200 done.
Batch 13/200 done.
Batch 14/200 done.
Batch 15/200 done.
Batch 16/200 done.
Batch 17/200 done.
Batch 18/200 done.
Batch 19/200 done.
Batch 20/200 done.
Batch 21/200 done.
Batch 22/200 done.
Batch 23/200 done.
Batch 24/200 done.
Batch 25/200 done.
Batch 26/200 done.
Batch 27/200 done.
Batch 28/200 done.
Batch 29/200 done.
Batch 30/200 done.
Batch 31/200 done.
Batch 32/200 done.
Batch 33/200 done.
Batch 34/200 done.
Batch 35/200 done.
Batch 36/200 done.
Batch 37/200 done.
Batch 38/200 done.
Batch 39/200 done.
Batch 40/200 done.
Batch 41/200 done.
Batch 42/200 done.
Batch 43/200 done.
Batch 44/200 done.
Batch 45/200 done.
Batch 46/200 done.
Batch 47/200 done.
Batch 48/200 done.
Batch 49/200 done.
Batch 50/200 done.
Batch 51/200 

### Saving the arrays to be used later 

In order to keep the data files relatively small (> 100MB's) for easier version controlling, the X_data is cut into 4 chunks to be concatenated later

In [ ]:
x1 = X_data[:50000]
x2 = X_data[50000:100000]
x3 = X_data[100000:150000]
x4 = X_data[150000:] 
np.save('X1.npy', x1) 
np.save('X2.npy', x2) 
np.save('X3.npy', x3)
np.save('X4.npy', x4)
np.save(file='y.npy', arr=y_data) 